training-data-analyst/courses/machine_learning/deepdive2/text_classification/labs/keras_for_text_classification.ipynb

## counts of each column

In [ ]:
dataframe.{column_name}.value_counts()

## Integerize the texts

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(titles_df.title)

In [ ]:
sequences = tokenizer.texts_to_sequences(texts) 
padded_sequences = pad_sequences(sequences, max_len, padding='post') 

In [ ]:
VOCAB_SIZE = len(tokenizer.index_word)
DATASET_SIZE = tokenizer.document_count
MAX_LEN = max(len(sequence) for sequence in integerized_titles)

In [ ]:
CLASSES = {
    'github': 0,
    'nytimes': 1,
    'techcrunch': 2
}
N_CLASSES = len(CLASSES)

# https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical
classes = [CLASSES.get(m) for m in sources]
one_hots = to_categorical(classes)

## Build DNN

In [ ]:
# https://github.com/keras-team/keras/issues/9637
# padding(=0)를 tokenized vocab에 포함하기 위해 Embedding layer의 input_dim=VOCAB_SIZE에 +1
Embedding(VOCAB_SIZE+1, embed_dim, input_length=MAX_LEN),  
Lambda(lambda x: tf.reduce_mean(x, axis=1)),
Dense(N_CLASSES, activation='softmax')

In [ ]:
## Show execute time of cell in Jupyter notebook
%%time

## code

## Build RNN

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
# the padded words (represented by a zero) are ignored by this and the subsequent layers.
Embedding(VOCAB_SIZE+1, embed_dim, input_length=MAX_LEN, mask_zero=True),
GRU(units),
Dense(N_CLASSES, activation='softmax')

## input dataset이 one-hot encoded 형태이고, loss function으로 sparse_categorical_crossentropy을 사용했을 때 발생하는 에러

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  logits and labels must have the same first dimension, got logits shape [300,3] and labels shape [900]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <timed exec>:18) ]]
	 [[gradient_tape/sequential_3/embedding_3/embedding_lookup/Reshape/_40]]
  (1) Invalid argument:  logits and labels must have the same first dimension, got logits shape [300,3] and labels shape [900]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <timed exec>:18) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_20316]

Function call stack:
train_function -> train_function


## Build CNN

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
Embedding(VOCAB_SIZE+1, embed_dim, input_length=MAX_LEN, mask_zero=True),
Conv1D(filters, ksize, strides=strides, activation='relu'),
Flatten(),
Dense(N_CLASSES, activation='softmax')